# Volby

Nedavano byly volby. Navic to byly komunalni, coz prinasi zajimave challenge pro cistotu dat.  
Chci se podivat na nekolik veci. Zajima me Strana Zelenych, chci se podivat:  
 - Jak se vyvinul pocet hlasu pro SZ mezi 2018 & 2022
 - Kde se ty hlasy vzaly, kam se pripadne prelily mezi temito lety?

Stahneme package na stahovani ceskych verejnych dat:

In [1]:
!git clone https://github.com/kokes/od.git

Cloning into 'od'...
remote: Enumerating objects: 2405, done.
remote: Counting objects: 100% (646/646), done.
remote: Compressing objects: 100% (192/192), done.
remote: Total 2405 (delta 533), reused 495 (delta 453), pack-reused 1759
Receiving objects: 100% (2405/2405), 550.20 KiB | 1.80 MiB/s, done.
Resolving deltas: 100% (1461/1461), done.


Stahneme volby pro data. U toho vytvorime novy environment, s potrebnymi zavislostmi.

In [2]:
!python3 -m venv .venv
!. .venv/bin/activate
!pip3 install -r od/requirements.txt
!python3 od/main.py volby

  Using cached dbfread-2.0.7-py2.py3-none-any.whl (20 kB)
  Using cached lxml-4.9.1.tar.gz (3.4 MB)
     |████████████████████████████████| 2.0 MB 3.8 MB/s eta 0:00:01
  Using cached cssselect-1.1.0-py2.py3-none-any.whl (16 kB)
Using legacy 'setup.py install' for lxml, since package 'wheel' is not installed.
    Running setup.py install for lxml ... done
You should consider upgrading via the '/opt/homebrew/Caskroom/miniforge/base/bin/python -m pip install --upgrade pip' command.
volby
=====


Jake datasety jsou k dispozici?

In [5]:
import os
os.listdir('csv/volby/komunalni_vysledky_obce')

['2006-10-20_KV2006reg20140909_kvros.csv',
 '2022-09-23_KV2022reg20220925b_kvros.csv',
 '2010-10-15_KV2010reg20140903_kvros.csv',
 '2014-10-10_KV2014reg20141014_kvros.csv',
 '2018-10-05_KV2018reg20181008_kvros.csv']

In [6]:
import pandas as pd

In [8]:
vysledky_2022 = pd.read_csv('csv/volby/komunalni_vysledky_obce/2022-09-23_KV2022reg20220925b_kvros.csv')

Zakladni info o datasetu

In [11]:
vysledky_2022.describe()

,OKRES,KODZASTUP,COBVODU,POR_STR_HL,OSTRANA,VSTRANA,POCSTR_SLO,HLASY_STR,PROCHLSTR,MAND_STR
count,23270.000000,23270.000000,23270.000000,23270.000000,23270.000000,23270.000000,23270.000000,2.326100e+04,23261.000000,23261.000000
mean,4837.019381,562384.995488,1.000258,3.778857,751.148388,177.767942,1.124022,4.615095e+03,50.027672,2.655948
std,1847.827993,23575.657337,0.016056,3.159232,295.571054,281.743913,0.388723,6.696738e+04,32.918508,2.731048
min,1100.000000,500011.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000e+00,0.000000,0.000000
25%,3107.000000,545431.000000,1.000000,1.000000,801.000000,80.000000,1.000000,7.600000e+01,20.860000,1.000000
50%,5204.000000,563633.000000,1.000000,3.000000,808.000000,90.000000,1.000000,4.350000e+02,44.580000,1.000000
75%,6201.000000,581963.500000,1.000000,5.000000,901.000000,90.000000,1.000000,1.616000e+03,77.770000,4.000000
max,8106.000000,599999.000000,2.000000,29.000000,1628.000000,1628.000000,6.000000,5.828523e+06,234.220000,25.000000


In [10]:
vysledky_2022.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23270 entries, 0 to 23269
Data columns (total 16 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   DATUM       23270 non-null  object 
 1   OKRES       23270 non-null  int64  
 2   KODZASTUP   23270 non-null  int64  
 3   NAZEVZAST   23270 non-null  object 
 4   COBVODU     23270 non-null  int64  
 5   POR_STR_HL  23270 non-null  int64  
 6   OSTRANA     23270 non-null  int64  
 7   VSTRANA     23270 non-null  int64  
 8   NAZEVCELK   23270 non-null  object 
 9   ZKRATKAO30  23270 non-null  object 
 10  ZKRATKAO8   23270 non-null  object 
 11  POCSTR_SLO  23270 non-null  int64  
 12  SLOZENI     23270 non-null  object 
 13  HLASY_STR   23261 non-null  float64
 14  PROCHLSTR   23261 non-null  float64
 15  MAND_STR    23261 non-null  float64
dtypes: float64(3), int64(7), object(6)
memory usage: 2.8+ MB



```
DATUM       datum
OKRES       numericke oznaceni okresu v ciselniku
KODZASTUP   kod zastupitelstva, do ktereho kandiduje
NAZEVZAST   nazev zastupitelstva
COBVODU     cislo volbeniho obvodu
POR_STR_HL  poradi strany v hlasovacich listcich
OSTRANA     ? asi identifikator strany?
VSTRANA     ? -- || --
NAZEVCELK   nazev celku
ZKRATKAO30  zkratka kandidujiho celku
ZKRATKAO8   jeste vetsi zkratka kandidujiho celku
POCSTR_SLO  pocet stran v celku
SLOZENI     slozeni celku pres identifikatory stran
HLASY_STR   pocet hlasu pro celek
PROCHLSTR   procentualni vysledek
MAND_STR    pocet mandatu
```

In [49]:
zeleni_2022 = vysledky_2022.query('SLOZENI.str.contains("005")')

In [50]:
zeleni_2022.describe()

,OKRES,KODZASTUP,COBVODU,POR_STR_HL,OSTRANA,VSTRANA,POCSTR_SLO,HLASY_STR,PROCHLSTR,MAND_STR
count,113.000000,113.000000,113.0,113.000000,113.000000,113.000000,113.000000,113.000000,113.000000,113.000000
mean,4527.203540,553622.327434,1.0,5.168142,684.185841,684.185841,2.460177,31481.929204,13.631416,2.858407
std,2136.088532,25567.146013,0.0,3.070315,537.255635,537.255635,0.935752,67240.391229,10.472428,2.434379
min,1100.000000,500089.000000,1.0,1.000000,5.000000,5.000000,1.000000,369.000000,1.650000,0.000000
25%,3102.000000,539864.000000,1.0,3.000000,364.000000,364.000000,2.000000,3855.000000,6.500000,1.000000
50%,4207.000000,554529.000000,1.0,5.000000,501.000000,501.000000,2.000000,9579.000000,10.570000,2.000000
75%,6202.000000,569810.000000,1.0,7.000000,1312.000000,1312.000000,3.000000,29811.000000,16.440000,4.000000
max,8106.000000,599549.000000,1.0,15.000000,1623.000000,1623.000000,5.000000,475968.000000,61.110000,12.000000


Na prvni dobrou z tehle tabulky vidim:
- Zeleni, jako strana, kandidovali v 113 celcich
- Tyto strany ziskaly prumerne 2.8 mandatu
- Nejvice takova strana ziskala 12 mandatu
- Nejvice take ziskali 61 procent

To me ted vede k nasledujicim otazkam:
 - kde ziskali nejvice mandatu, procent?
 - z kolika stran se takove celky skladaly?
 - kolik ziskali zeleni mandatu?
 - existuje nejaky vztah: #stran v koalici + velikost obce na pocet mandatu?
 - existuje nejaky vztah: vek kandidata/velikost obce na mandat?
 
To bude potrebovat nove datasety, a nejakou zakladni manipulaci s daty.

In [28]:
obce_2022 = pd.read_csv('csv/volby/komunalni_obce/2022-09-23_KV2022reg20220925b_kvrzcoco.csv')
obce_2022

,DATUM,KRAJ,OKRES,TYPZASTUP,DRUHZASTUP,KODZASTUP,NAZEVZAST,OBEC,NAZEVOBCE,ORP,CPOU,REGURAD,OBVODY,COBVODU,MANDATY,POCOBYV,TYPDUVODU,POCET_VS,STAV_OBCE
0,2022-09-23,7200,7204,1,1,500011,Želechovice nad Dřevnicí,500011,Želechovice nad Dřevnicí,7213,662,585068,0,1,15,1840,1,5,0
1,2022-09-23,7100,7105,1,1,500020,Petrov nad Desnou,500020,Petrov nad Desnou,7111,860,523704,0,1,11,1230,1,7,0
2,2022-09-23,8100,8104,1,1,500046,Libhošť,500046,Libhošť,8115,792,599191,0,1,15,1723,1,1,0
3,2022-09-23,1100,1100,2,5,500054,Praha 1,500054,Praha 1,1000,1,500054,0,1,27,26617,1,9,0
4,2022-09-23,7200,7203,1,1,500062,Krhová,500062,Krhová,7210,871,545058,0,1,15,2024,1,4,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6525,2022-09-23,8100,8104,1,6,599930,Suchdol nad Odrou,599930,Suchdol nad Odrou,8115,792,599930,0,1,15,2668,1,4,0
6526,2022-09-23,8100,8104,1,2,599948,Štramberk,599948,Štramberk,8112,791,599565,0,1,15,3467,1,5,0
6527,2022-09-23,8100,8104,1,1,599956,Tichá,599956,Tichá,8105,789,599344,0,1,11,1875,1,3,0
6528,2022-09-23,8100,8104,1,1,599964,Tísek,599964,Tísek,8101,788,599247,0,1,9,980,1,5,0


In [51]:
kandidati_2022 = pd.read_csv("csv/volby/komunalni_kandidati/2022-09-23_KV2022reg20220925b_kvrk.csv")
zeleni_kandidati_2022 = kandidati_2022.query('PSTRANA==5 | NSTRANA==5')

In [65]:
zeleni_kandidati_2022.query('MANDAT')

,DATUM,OKRES,KODZASTUP,COBVODU,POR_STR_HL,OSTRANA,PORCISLO,JMENO,PRIJMENI,TITULPRED,...,PSTRANA,NSTRANA,PLATNOST,POCHLASU,PRESKOCENI,POCHL_PRES,POCPROCVSE,MANDAT,PORADIMAND,PORADINAHR
277,2022-09-23,1100,500054,1,5,1373,3,Petr,Kučera,Mgr.,...,5.0,80.0,A,1827,N,NaN,3.93,True,3,0
279,2022-09-23,1100,500054,1,5,1373,5,Barbora,Kosíková,Mgr.,...,5.0,80.0,A,1744,N,NaN,3.76,True,5,0
280,2022-09-23,1100,500054,1,5,1373,6,Pavel,Marc,JUDr.,...,5.0,80.0,A,1761,N,NaN,3.79,True,6,0
1089,2022-09-23,1100,500097,1,10,364,1,Ondřej,Rut,Mgr.,...,5.0,5.0,A,1913,A,NaN,4.25,True,1,0
1091,2022-09-23,1100,500097,1,10,364,3,Matěj Michalk,Žaloudek,Ing. arch.,...,5.0,5.0,A,1754,A,NaN,3.89,True,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
167539,2022-09-23,7103,589250,1,2,1612,6,Martin,Hájek,Mgr.,...,5.0,5.0,A,2264,A,NaN,3.34,True,2,0
173941,2022-09-23,7202,592064,1,2,902,1,Roman,Jílek,Ing.,...,5.0,80.0,A,525,A,NaN,12.97,True,1,0
193012,2022-09-23,8104,599191,1,1,1351,1,Ondřej,Syrovátka,NaN,...,5.0,5.0,A,1556,A,NaN,4.89,True,1,0
193015,2022-09-23,8104,599191,1,1,1351,4,Lucie,Jančařiková,NaN,...,5.0,5.0,A,1172,N,NaN,3.68,True,5,0


**Zeleni ziskali 142 mandatu.**  
Tohle cislo bohuzel nesedi na oficialni prohlaseni strany:  
*V aktuálních komunálních volbách jsme kandidovali ve 144 obcí a měst,  
po volbách máme 162 zastupitelských mandátů.*

Tady by sly delat dalsi operace. Zde je filtrovani kandidatu je jednoduche, delane pouze podle kodu strany.  
Myslim si, ze do oficialniho poctu se pouzivaji take nezavisli kandidati z ruznych koalic, kteri ale maji jiny kod.  
Ti by sli vyfiltrovat skrz nazev strany (obsahuje 'zelen').  
coz by ale zas mohlo prinest jine problemy (koukam na tebe, demokraticka strano zelenych)  
Pro ted ale budu pracovat uz jen s timto seznamem.

In [91]:
obce_2022

,DATUM,KRAJ,OKRES,TYPZASTUP,DRUHZASTUP,KODZASTUP,NAZEVZAST,OBEC,NAZEVOBCE,ORP,CPOU,REGURAD,OBVODY,COBVODU,MANDATY,POCOBYV,TYPDUVODU,POCET_VS,STAV_OBCE
0,2022-09-23,7200,7204,1,1,500011,Želechovice nad Dřevnicí,500011,Želechovice nad Dřevnicí,7213,662,585068,0,1,15,1840,1,5,0
1,2022-09-23,7100,7105,1,1,500020,Petrov nad Desnou,500020,Petrov nad Desnou,7111,860,523704,0,1,11,1230,1,7,0
2,2022-09-23,8100,8104,1,1,500046,Libhošť,500046,Libhošť,8115,792,599191,0,1,15,1723,1,1,0
3,2022-09-23,1100,1100,2,5,500054,Praha 1,500054,Praha 1,1000,1,500054,0,1,27,26617,1,9,0
4,2022-09-23,7200,7203,1,1,500062,Krhová,500062,Krhová,7210,871,545058,0,1,15,2024,1,4,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6525,2022-09-23,8100,8104,1,6,599930,Suchdol nad Odrou,599930,Suchdol nad Odrou,8115,792,599930,0,1,15,2668,1,4,0
6526,2022-09-23,8100,8104,1,2,599948,Štramberk,599948,Štramberk,8112,791,599565,0,1,15,3467,1,5,0
6527,2022-09-23,8100,8104,1,1,599956,Tichá,599956,Tichá,8105,789,599344,0,1,11,1875,1,3,0
6528,2022-09-23,8100,8104,1,1,599964,Tísek,599964,Tísek,8101,788,599247,0,1,9,980,1,5,0


In [95]:
zeleni_kandidati_2022[['KODZASTUP', 'OKRES']].value_counts()

KODZASTUP  OKRES
563889     5103     39
505927     8105     39
567027     4205     33
582786     6202     29
535419     2107     28
                    ..
565971     4204      1
538728     2109      1
572781     5202      1
572926     5202      1
599565     8104      1
Length: 155, dtype: int64

In [72]:
zeleni_zastupitelstvo_mandaty = zeleni_kandidati_2022.groupby(['KODZASTUP']).agg({'MANDAT': 'sum', 'PRIJMENI':'count'})
zeleni_zastupitelstvo_mandaty = zeleni_zastupitelstvo_mandaty.rename(columns={'PRIJMENI': 'POCET_KANDIDUJICICH_ZELENYCH'})

In [73]:
zeleni_zastupitelstvo_mandaty

,MANDAT,POCET_KANDIDUJICICH_ZELENYCH
KODZASTUP,,
500054,3,7
500089,0,14
500097,2,22
500119,5,19
500143,0,19
...,...,...
599026,0,1
599191,3,13
599212,0,1


Pocet obci, kde zeleni kandidovali, vychazi na 155. To je vice, nez oficialni cislo.  
Ocekavam, ze zde se kryji ctvrti/magistrat, jako tomu napriklad bylo v Praze. Brzy zkontrolujeme.

,DATUM,KRAJ,OKRES,TYPZASTUP,DRUHZASTUP,KODZASTUP,NAZEVZAST,OBEC,NAZEVOBCE,ORP,CPOU,REGURAD,OBVODY,COBVODU,MANDATY,POCOBYV,TYPDUVODU,POCET_VS,STAV_OBCE
1687,2022-09-23,3100,3101,1,2,544779,Lišov,544779,Lišov,3102,217,544779,1,2,21,4508,1,6,0
1689,2022-09-23,3100,3101,1,2,544779,Lišov,544779,Lišov,3102,217,544779,2,2,6,4508,1,6,0


In [99]:
obce_2022.query('KODZASTUP==554782')

,DATUM,KRAJ,OKRES,TYPZASTUP,DRUHZASTUP,KODZASTUP,NAZEVZAST,OBEC,NAZEVOBCE,ORP,CPOU,REGURAD,OBVODY,COBVODU,MANDATY,POCOBYV,TYPDUVODU,POCET_VS,STAV_OBCE
2643,2022-09-23,1100,1100,1,4,554782,Praha hl.m.,500054,Praha 1,1000,1,554782,0,1,65,1275406,1,22,0
2644,2022-09-23,1100,1100,1,4,554782,Praha hl.m.,500089,Praha 2,1000,2,554782,0,1,65,1275406,1,22,0
2645,2022-09-23,1100,1100,1,4,554782,Praha hl.m.,500097,Praha 3,1000,3,554782,0,1,65,1275406,1,22,0
2646,2022-09-23,1100,1100,1,4,554782,Praha hl.m.,500119,Praha 4,1000,4,554782,0,1,65,1275406,1,22,0
2647,2022-09-23,1100,1100,1,4,554782,Praha hl.m.,500143,Praha 5,1000,11,554782,0,1,65,1275406,1,22,0
2648,2022-09-23,1100,1100,1,4,554782,Praha hl.m.,500178,Praha 6,1000,21,554782,0,1,65,1275406,1,22,0
2649,2022-09-23,1100,1100,1,4,554782,Praha hl.m.,500186,Praha 7,1000,27,554782,0,1,65,1275406,1,22,0
2650,2022-09-23,1100,1100,1,4,554782,Praha hl.m.,500208,Praha 8,1000,29,554782,0,1,65,1275406,1,22,0
2651,2022-09-23,1100,1100,1,4,554782,Praha hl.m.,500216,Praha 9,1000,33,554782,0,1,65,1275406,1,22,0
2652,2022-09-23,1100,1100,1,4,554782,Praha hl.m.,500224,Praha 10,1000,46,554782,0,1,65,1275406,1,22,0


In [106]:
model_data = zeleni_zastupitelstvo_mandaty.reset_index()
model_data = model_data.merge(obce_2022[['KODZASTUP', 'TYPZASTUP', 'DRUHZASTUP', 'NAZEVZAST', 'MANDATY', 'POCOBYV']], on='KODZASTUP', how='inner')
# jednotlive casti pri volbach na i.e. velkou Prahu jsou zastoupeny v datech, ale samotna data jsou validni
model_data = model_data.drop_duplicates()

In [107]:
model_data

,KODZASTUP,MANDAT,POCET_KANDIDUJICICH_ZELENYCH,TYPZASTUP,DRUHZASTUP,NAZEVZAST,MANDATY,POCOBYV
0,500054,3,7,2,5,Praha 1,27,26617
1,500089,0,14,2,5,Praha 2,35,46101
2,500097,2,22,2,5,Praha 3,35,72809
3,500119,5,19,2,5,Praha 4,45,126070
4,500143,0,19,2,5,Praha 5,41,85687
...,...,...,...,...,...,...,...,...
262,599026,0,1,1,1,Slapsko,7,149
263,599191,3,13,1,2,Nový Jičín,29,22813
264,599212,0,1,1,1,Bartošovice,15,1705
265,599549,0,3,1,2,Klimkovice,17,4469


In [112]:
vysledky_2022

,DATUM,OKRES,KODZASTUP,NAZEVZAST,COBVODU,POR_STR_HL,OSTRANA,VSTRANA,NAZEVCELK,ZKRATKAO30,ZKRATKAO8,POCSTR_SLO,SLOZENI,HLASY_STR,PROCHLSTR,MAND_STR
0,2022-09-23,7204,500011,Želechovice nad Dřevnicí,1,1,902,90,POCTIVÁ OBEC ŽELECHOVICE NAD DŘEVNICÍ,POCTIVÁ OBEC ŽELECHOVICE N. D.,SNK,1,080,4690.0,39.88,6.0
1,2022-09-23,7204,500011,Želechovice nad Dřevnicí,1,2,470,470,SPOLEČNĚ PRO LEPŠÍ ŽELECHOVICE,SPOLEČNĚ PRO LEPŠÍ ŽELECHOVICE,STAN+NK,2,"080,166",3020.0,25.68,4.0
2,2022-09-23,7204,500011,Želechovice nad Dřevnicí,1,3,901,90,STRANA ZA ROZVOJ OBCE,STRANA ZA ROZVOJ OBCE,SNK,1,080,1576.0,13.40,2.0
3,2022-09-23,7204,500011,Želechovice nad Dřevnicí,1,4,371,371,Obec pro všechny generace,Obec pro všechny generace,ČSSD+NK,2,"007,080",1532.0,13.02,2.0
4,2022-09-23,7204,500011,Želechovice nad Dřevnicí,1,5,1,1,KDU-ČSL,KDU-ČSL,KDU-ČSL,1,001,940.0,7.99,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23265,2022-09-23,8104,599964,Tísek,1,5,901,90,Hasiči pro Tísek,Hasiči pro Tísek,SNK,1,080,927.0,24.14,2.0
23266,2022-09-23,8104,599999,Trojanovice,1,1,903,90,PRO ZMĚNU,PRO ZMĚNU,SNK,1,080,787.0,7.41,0.0
23267,2022-09-23,8104,599999,Trojanovice,1,2,1,1,KDU-ČSL,KDU-ČSL,KDU-ČSL,1,001,823.0,6.34,0.0
23268,2022-09-23,8104,599999,Trojanovice,1,3,902,90,Naše Trojanovice,Naše Trojanovice,SNK,1,080,2905.0,22.39,3.0


In [127]:
zelene_strany = (vysledky_2022
                 .query('NAZEVCELK.str.lower().str.contains("zelen") | SLOZENI.str.contains("005")')
                 [['KODZASTUP', 'HLASY_STR', 'SLOZENI', 'NAZEVCELK']])
zelene_strany['N_STRAN'] = zelene_strany['SLOZENI'].str.count(',')
zelene_strany['N_STRAN'] += 1
zelene_strany

,KODZASTUP,HLASY_STR,SLOZENI,NAZEVCELK,N_STRAN
31,500089,20233.0,"001,005,007",Dvojka solidární a zelená,3
47,500097,45003.0,"005,080",Zelení Praha 3,2
60,500119,273219.0,"005,080",Praha 4 sobě – sousedé a nezávislí,2
70,500143,41873.0,"001,005","KDU-ČSL, Zelení a Nezávislí",2
82,500160,607.0,080,Zelená pro Libavou,1
...,...,...,...,...,...
22843,598925,1622.0,"005,080",ZELENÍ V ALBRECHTICÍCH,2
23009,599191,31776.0,"005,166,720,721","Strana zelených s Piráty, TOP 09 a STAN",4
23048,599280,28.0,080,Miroslav Zelenka,1
23148,599549,2134.0,"005,007,080",Budoucnost pro Klimkovice - sdružení nezávislý...,3


In [137]:
model_data = model_data.merge(zelene_strany[['KODZASTUP', 'N_STRAN']], on='KODZASTUP')

In [138]:
model_data['ZELENI_SHARE'] = model_data['POCET_KANDIDUJICICH_ZELENYCH']/model_data['MANDATY']

In [132]:
from statsmodels.regression.linear_model import OLS

In [144]:
variables_to_use = [
#     'ZELENI_SHARE',
    'N_STRAN',
    'MANDATY'
]

In [145]:
model = OLS(endog=model_data['MANDAT'], exog=model_data[variables_to_use])
results = model.fit()

In [146]:
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                 MANDAT   R-squared (uncentered):                   0.345
Model:                            OLS   Adj. R-squared (uncentered):              0.334
Method:                 Least Squares   F-statistic:                              31.35
Date:                Tue, 11 Oct 2022   Prob (F-statistic):                    1.15e-11
Time:                        23:19:33   Log-Likelihood:                         -214.68
No. Observations:                 121   AIC:                                      433.4
Df Residuals:                     119   BIC:                                      438.9
Df Model:                           2                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
N_STRAN        0.2023      0.105      1.928      0.056      -0.005       0.410
MANDATY        0.0191      0.010      1.982      0.050    2.27e-05       0.038
==============================================================================
Omnibus:                       55.169   Durbin-Watson:                   2.089
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              154.046
Skew:                           1.764   Prob(JB):                     3.54e-34
Kurtosis:                       7.256   Cond. No.                         22.9
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""